In [ ]:
import pandas as pd

def imputar_categoricas(df, columnas, umbral_nulos_alto=0.20, umbral_nulos_bajo=0.05,
                        umbral_moda_bajo=0.50, umbral_moda_medio=0.60, umbral_ventaja=0.20,
                        etiqueta_unknown="Unknown"):
    """
    Imputa nulos en variables categóricas siguiendo reglas simples y justificables en EDA.

    Reglas (resumen):
    1) Si el % de nulos es ALTO (> umbral_nulos_alto, por defecto 20%) → imputar con etiqueta_unknown.
       - Motivo: imputar por moda con muchos nulos puede inventar demasiada información.

    2) Si el % de nulos es BAJO (<= umbral_nulos_bajo, por defecto 5%) o MEDIO (entre 5% y 20%):
       - Solo imputamos con la MODA si hay una categoría realmente dominante.
       - Para considerar "dominante" exigimos 2 condiciones:
         a) La moda supera un umbral según el % de nulos:
            - Si nulos BAJOS: moda >= umbral_moda_bajo (50% por defecto)
            - Si nulos MEDIOS: moda >= umbral_moda_medio (60% por defecto)
         b) La moda tiene suficiente ventaja sobre la 2ª categoría:
            - (pct_moda - pct_segunda) >= umbral_ventaja (20 puntos porcentuales por defecto)

       Si no se cumple lo anterior → imputar con etiqueta_unknown.

    Parámetros
    ----------
    df : pandas.DataFrame
        DataFrame de entrada (se modifica y también se devuelve).
    columnas : list[str]
        Lista de columnas categóricas a imputar.
    umbral_nulos_alto : float, default 0.20
        Por encima de este porcentaje de nulos se usa etiqueta_unknown.
    umbral_nulos_bajo : float, default 0.05
        Hasta este porcentaje de nulos se considera "bajo".
    umbral_moda_bajo : float, default 0.50
        Umbral mínimo de la moda si los nulos son bajos.
    umbral_moda_medio : float, default 0.60
        Umbral mínimo de la moda si los nulos son medios (5%-20%).
    umbral_ventaja : float, default 0.20
        Ventaja mínima (en proporción, 0.20 = 20 puntos porcentuales) entre la moda y la 2ª categoría.
    etiqueta_unknown : str, default "Unknown"
        Etiqueta para imputar cuando no se quiere usar la moda.

    Returns
    -------
    pandas.DataFrame
        El mismo DataFrame con los nulos imputados en las columnas indicadas.
    """

    total = len(df)

    for col in columnas:
        print(f"\n📌 Analizando columna: {col}")
        
        # Si la columna no existe, evitamos error y seguimos
        if col not in df.columns:
            print(f"❌ La columna {col} no existe en el DataFrame. Se omite.")
            continue

        nulos = df[col].isnull().sum()
        porcentaje_nulos = nulos / total if total > 0 else 0

        print(f"   → Nulos: {nulos} de {total} ({porcentaje_nulos:.2%})")

        # Caso 1: muchos nulos -> Unknown directamente
        if porcentaje_nulos > umbral_nulos_alto:
            print(
                f"   🔴 Porcentaje de nulos > {umbral_nulos_alto:.0%} "
                f"→ se crea la categoría '{etiqueta_unknown}'"
            )
            df[col] = df[col].fillna(etiqueta_unknown)
            continue
        
        # Calculamos frecuencias (sin nulos) para decidir si hay moda dominante
        valores = df[col].value_counts(dropna=True)

        if len(valores) == 0:
            # No hay valores no nulos para decidir moda (columna vacía o todo nulo)
            print(
                f"   🔴 No hay valores no nulos para decidir moda "
                f"→ se crea la categoría '{etiqueta_unknown}'"
            )
            df[col] = df[col].fillna(etiqueta_unknown)
            continue

        # Frecuencia y porcentaje de la moda
        primero = valores.iloc[0]
        pct_primero = primero / total # proporción sobre el total de filas

        # Frecuencia y porcentaje de la segunda categoría (si existe)
        if len(valores) > 1:
            segundo = valores.iloc[1]
            pct_segundo = segundo / total
        else:
            pct_segundo = 0.0

        ventaja = pct_primero - pct_segundo # ventaja de la moda frente a la 2ª (en proporción)

        print(f"   → Moda: {valores.index[0]} ({pct_primero:.2%})")
        print(f"   → 2ª categoría: {valores.index[1] if len(valores) > 1 else 'No existe'} ({pct_segundo:.2%})")
        print(f"   → Ventaja de la moda: {ventaja:.2%}")

        # Umbral de moda depende de si el % de nulos es bajo o medio
        if porcentaje_nulos <= umbral_nulos_bajo:
            umbral_moda = umbral_moda_bajo
            print(
                f"   → Nulos bajos (≤ {umbral_nulos_bajo:.0%}), "
                f"umbral de moda requerido: {umbral_moda:.0%}"
            )
        else:
            umbral_moda = umbral_moda_medio
            print(
                f"   → Nulos medios (> {umbral_nulos_bajo:.0%} y ≤ {umbral_nulos_alto:.0%}), "
                f"umbral de moda requerido: {umbral_moda:.0%}"
            )
        
        # Caso 2: imputar por moda solo si es dominante y con ventaja suficiente
        if (pct_primero >= umbral_moda) and (ventaja >= umbral_ventaja):
            moda = df[col].mode(dropna=True)[0]
            print(
                f"   🟢 La moda es dominante y con ventaja suficiente "
                f"(≥ {umbral_ventaja:.0%}) → se imputan nulos con '{moda}'"
            )
            df[col] = df[col].fillna(moda)
        else:
            print(
                f"   🟡 La moda NO es lo suficientemente dominante "
                f"→ se crea la categoría '{etiqueta_unknown}'"
            )
            df[col] = df[col].fillna(etiqueta_unknown)

    return df


In [ ]:
cols_cat = df.select_dtypes(include=["object", "category"]).columns.tolist()
cols_cat = [c for c in cols_cat if c.lower() != "attrition"]  # excluimos attrition

# normaliza falsos nulos solo en esas columnas
df[cols_cat] = df[cols_cat].replace(r'^\s*$', pd.NA, regex=True)

# imputación
df = imputar_categoricas(df, cols_cat)